In [27]:
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import pandas as pd
import requests
import urllib3
import seaborn as sns
from pandas import json_normalize
import datetime as dt
import pytz
import login as login


sns.set_theme()
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [31]:
from login import client_id, client_secret, refresh_token

client_id = login.client_id
client_secret = login.client_secret
refresh_token = login.refresh_token

ImportError: cannot import name 'client_id' from 'login' (/Users/gijsdesmet/.pyenv/versions/3.11.2/lib/python3.11/site-packages/login/__init__.py)

In [17]:
auth_url = 'https://www.strava.com/oauth/token'
activities_url = 'https://www.strava.com/api/v3/athlete/activities'

payload = {
    'client_id': f'96996',
    'client_secret': f'4506231e9b98cb9bc2074f0bc44df7eeccfe8ed9',
    'refresh_token': f'63c1edc5b126234baf675a783bbf29c1c5b99c9c',
    'grant_type': 'refresh_token',
    'f': 'json'
}

print('Requesting Token...\n')
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']

header = {'Authorization': 'Bearer ' + access_token}
print("Success!")

Requesting Token...

Success!


In [28]:
auth_url = 'https://www.strava.com/oauth/token'
activities_url = 'https://www.strava.com/api/v3/athlete/activities'

payload = {
    'client_id': f'{login.client_id}',
    'client_secret': f'{login.client_secret}',
    'refresh_token': f'{login.refresh_token}',
    'grant_type': 'refresh_token',
    'f': 'json'
}

print('Requesting Token...\n')
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']

header = {'Authorization': 'Bearer ' + access_token}
print("Success!")

AttributeError: module 'login' has no attribute 'client_id'

In [18]:
# The Strava API only supports 200 results per page. This function loops through each page until new_results is empty.
def loop_through_pages(page):
    # start at page ...
    page = page
    # set new_results to True initially
    new_results = True
    # create an empty array to store our combined pages of data in
    data = []
    while new_results:
        # Give some feedback
        print(f'You are requesting page {page} of your activities data ...')
        # request a page + 200 results
        get_strava = requests.get(activities_url, headers=header, params={'per_page': 200, 'page': f'{page}'}).json()
        # save the response to new_results to check if its empty or not and close the loop
        new_results = get_strava
        # add our responses to the data array
        data.extend(get_strava)
        # increment the page
        page += 1
    # return the combine results of our get requests
    return data

In [19]:
# call the function to loop through our strava pages and set the starting page at 1
my_dataset = loop_through_pages(1)

You are requesting page 1 of your activities data ...
You are requesting page 2 of your activities data ...
You are requesting page 3 of your activities data ...


In [22]:
# convert to pandas df
activities = json_normalize(my_dataset)

my_activities = pd.DataFrame(data=activities)

In [23]:
def convert_distance_measures(df):
    df['distance_miles'] = round(df['distance'] / 1609, 2)
    df['distance_km'] = round(df['distance'] / 1000, 2)
    df['average_speed_kph'] = round(df['average_speed'] * 3.6, 2)
    df['average_speed_mph'] = round(df['average_speed'] * 2.237, 2)
    return df

In [24]:
my_activities = convert_distance_measures(my_activities)

In [25]:
my_activities.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,...,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_cadence,distance_miles,distance_km,average_speed_kph,average_speed_mph
0,2,Evening Ride,26648.9,4037,4362,46.0,Ride,Ride,10.0,12114308928,...,26550196,1,a12114308928,ytpvHqgzU{BeFa@mFw@iCWWo@iBMw@AgA_AeEKiB@_Bh@i...,2,NaN,16.56,26.65,23.76,14.77
1,2,Evening Run,5917.4,2122,2122,0.0,Run,Run,NaN,12083544241,...,26550196,1,a12083544241,,2,73.7,3.68,5.92,10.04,6.24
2,2,Morning Run,8038.4,2965,2983,0.0,Run,Run,0.0,11908012633,...,26550196,1,a11908012633,cgcwHimtVYJc@`@q@z@u@n@u@h@YJs@~@k@T}@j@u@X[b@...,2,75.2,5.00,8.04,9.76,6.06
3,2,Afternoon Ride,13786.3,2179,2699,11.0,Ride,Ride,10.0,11894629657,...,26550196,1,a11894629657,wgsvHq_zUWu@Og@AmAGWMMc@AqAx@QR{A|@_@\GTUd@SXK...,2,NaN,8.57,13.79,22.78,14.15
4,2,Lunch Run,6807.4,3076,3266,2.0,Run,Run,0.0,11884995468,...,26550196,1,a11884995468,afnvHklxUDDH[AMDu@XWXKHQXQb@iAFc@\w@D_@Ks@@ILM...,2,72.5,4.23,6.81,7.97,4.95


In [86]:
# get unique Strava activity types
my_activities.type.unique()

array(['Ride', 'Run', 'Walk', 'AlpineSki', 'Workout', 'WeightTraining'],
      dtype=object)

In [87]:
my_run_activites = my_activities[my_activities['type'] == 'Run']

In [90]:
input_cols = ['name', 'distance_km', 'average_speed_kph', 'start_date_local', 'moving_time', 'elapsed_time', 'total_elevation_gain', 'average_heartrate']

input_list = my_run_activites[input_cols].head().values.tolist()

In [91]:
import vertexai
from vertexai.generative_models import GenerativeModel

vertexai.init(project='oi-data-science', location='europe-west1')
model = GenerativeModel("gemini-1.5-flash-001")

template = """
        You need to act as a running coach. 
        I will give you a list of the 5 last run activities of an athlete, and you need to define the next 3 recommended activities based on that. 
        Take into account elpased time, average heartrate, distance and speed.
        First, give some feedback about the last 5 activities, using encouraging words and being specific about the athlete's performance.
        I need you to be very specific about the next recommended activities and you should return them as bullet points:
            Name (type of workout):
            Duration:
            Target Speed (in km/h):
            Target Heart Rate:
            Perceived effort (score out of 10):

        Input: 

        {input_list}

    """

generated_text = model.generate_content(template.format(input_list=input_list)).candidates[0].content.parts[0].text
print(generated_text)

Okay, let's take a look at your recent runs! You've been putting in some great work, with a good balance of distance and intensity. I see you're consistently hitting those longer runs, and your speed is looking strong. You've also been mixing up your paces, which is key for overall fitness and preventing plateaus.

Here's a breakdown of your last five runs:

* **Evening Run:** You finished strong with a solid pace and good heart rate. This is a great way to end your day! 
* **Morning Run:** You crushed that longer distance with a good average speed!  This is a fantastic example of a well-executed endurance run.
* **Lunch Run:** This was a nice recovery run, allowing your body to rest and recover from the longer runs. 
* **Afternoon Run:**  You nailed this interval run! Your heart rate stayed elevated throughout, showcasing great speed and stamina. 
* **Night Run:** Another strong effort with a good pace and duration, proving your ability to handle longer runs with intensity. 

Keep up 